<a href="https://colab.research.google.com/github/omarfathyz8/Teeth-Classification/blob/main/TeethClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##  Imports

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import os
import zipfile
import tensorflow as tf
from tensorflow.keras import layers
import shutil

## Loading Datasets

#### Drive Mounting

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


#### Unzip Dataset Folder

In [ ]:
ZIP_PATH = "/content/drive/MyDrive/CellulaTechnologies/ComputerVision/TeethDataset.zip"
EXTRACT_DIR = "/content/TeethDataset"

In [ ]:
if not os.path.exists(EXTRACT_DIR):
    with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
        zip_ref.extractall(EXTRACT_DIR)

In [ ]:
print("Extracted to:", EXTRACT_DIR)

Extracted to: /content/TeethDataset


#### Data Directories

In [ ]:
DATA_DIR = "/content/TeethDataset/Teeth_Dataset"
TRAIN_DIR = os.path.join(DATA_DIR, "Training")
TEST_DIR = os.path.join(DATA_DIR, "Testing")
VAL_DIR = os.path.join(DATA_DIR, 'Validation')

In [ ]:
print('Training classes   :', os.listdir(TRAIN_DIR))
print('Testing classes    :', os.listdir(TEST_DIR))
print('Validation classes :', os.listdir(VAL_DIR))

Training classes   : ['MC', 'OC', 'OLP', 'Gum', 'OT', 'CaS', 'CoS']
Testing classes    : ['output_image.jpg', 'outputs', 'MC', 'OC', 'OLP', 'Gum', 'output', 'OT', 'CaS', 'CoS', 'out']
Validation classes : ['MC', 'OC', 'OLP', 'Gum', 'OT', 'CaS', 'CoS']


#### Remove Extra Classes from Test Folder

In [ ]:
# Allowed class names
allowed_classes = ['MC', 'OC', 'OLP', 'Gum', 'OT', 'CaS', 'CoS']

# Loop through test_dir and remove anything not in allowed_classes
for item in os.listdir(TEST_DIR):
    item_path = os.path.join(TEST_DIR, item)
    if item not in allowed_classes:
        if os.path.isdir(item_path):
            shutil.rmtree(item_path)  # remove folder
        else:
            os.remove(item_path)      # remove file
        print(f"Removed: {item}")

print("\nCleaned test directory successfully.\nClasses now:", os.listdir(TEST_DIR))

Removed: output_image.jpg
Removed: outputs
Removed: output
Removed: out

Cleaned test directory successfully.
Classes now: ['MC', 'OC', 'OLP', 'Gum', 'OT', 'CaS', 'CoS']


## Data Preprocessing

#### Prepare Datasets

In [ ]:
IMG_SIZE = (256, 256) # the images size is already 256*256
BATCH_SIZE = 32

train_ds = tf.keras.utils.image_dataset_from_directory(
    TRAIN_DIR,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    VAL_DIR,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    TEST_DIR,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

Found 3087 files belonging to 7 classes.
Found 1028 files belonging to 7 classes.
Found 1028 files belonging to 7 classes.


#### Normalization

In [ ]:
normalization_layer = layers.Rescaling(1/255.0)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds   = val_ds.map(lambda x, y: (normalization_layer(x), y))
test_ds  = test_ds.map(lambda x, y: (normalization_layer(x), y))

#### Augmentation

In [ ]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.05),
    layers.RandomZoom(0.1),
])

In [ ]:
# Apply augmentation only on the training dataset
train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y))